In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):

    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Import Libraries 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer

# Precision, recall, f1-score for all the classes
from sklearn.metrics import classification_report, confusion_matrix

#Image data gen for possible data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Keras and relevant libraries for modeling 
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

# Load Data 

In [ ]:
df_train = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
df_test = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [ ]:
#Checking out info 
print(df_train.info())
#Exploring data 
df_train.head()

In [ ]:
#Checking out null values
print(df_train.isnull().sum())

There is no null values and the pixel appears to be ready for further analysis.

# EDA and Data Preprocessing

We need to do three task in order to make our data ready for modeling.
1.  **Reshaping:** As we need our image in shape of total_data x 28 x 28 x 1 where 1 represent the channels
2.  **Normalization:**  To get our data between 0-1 and this will result in incrreasing the speed of CNN.
3.  **Label encoding:** our labels using Onehot Encoder or Label Binirizer 

In [ ]:
#Reshaping
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)

Y_train = df_train['label'].values
Y_test = df_test['label'].values

X_train = df_train.drop(["label"],axis=1).values
X_test = df_test.drop(["label"],axis=1).values

print(X_train.shape)
print(X_test.shape)

training_images = X_train.reshape(-1,28,28,1)
testing_images = X_test.reshape(-1,28,28,1)

print(training_images.shape)
print(testing_images.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
#Normalizing 
training_images  = training_images / 255.0
testing_images = testing_images / 255.0
print(training_images.shape)
print(testing_images.shape)

Lets see the first 8 images with their required labels.

In [ ]:
f, ax = plt.subplots(2,4)
f.set_size_inches(8,8)

k = 0
for i in range(2):
    for j in range(4):
        ax[i,j].set_xlabel(chr(Y_test[k] + 65))
        ax[i,j].imshow(testing_images[k].reshape(28,28)
                       ,cmap='gray')
        k += 1
    plt.tight_layout()

In [ ]:
#Label Encoding 
label_binrizer = LabelBinarizer()
Y_train = label_binrizer.fit_transform(Y_train)
Y_test = label_binrizer.fit_transform(Y_test)
print('First image label after encoding: ',Y_train[0])

lets check out the labels mentioned in the description

In [ ]:
#Using unique to find out different numeric intrepretation of symbols
label_unique = df_train.label.values
unique_val = np.array(label_unique)
np.unique(unique_val)

In [ ]:
#Count plot to show freauency of different labels
plt.figure(figsize=(10,5))
sns.countplot(label_unique, color = 'black')

# Data Augmentation 

In [ ]:
#Now create Image generator object for train images 
train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, 
                             height_shift_range=0.2, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip = True, 
                             fill_mode='nearest')


In [ ]:
#for validation data 
validation_datagen = ImageDataGenerator()

In [ ]:
#Checking shape of training and testing data 
print(training_images.shape)
print(testing_images.shape)

# Modeling 

In [ ]:
# Making layers
model= Sequential()


model.add(Conv2D(filters= 128 , kernel_size=(3,3), padding = 'same', 
                  activation = 'relu' , input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size=(2,2), strides = 2 , 
                    padding = 'same'))

model.add(Conv2D(filters=64, kernel_size=(3,3) , padding = 'same',
                 activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2), strides = 2 , 
                    padding = 'same'))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding = 'same',
                 activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides = 2 , 
                    padding = 'same'))

model.add(Flatten())

model.add(Dense(512,kernel_initializer="he_normal", activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(24, activation = 'softmax'))


          


In [ ]:
#Compiling Model 
model.compile(optimizer= 'adam' ,loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.summary()

In [ ]:
# Train the Model

history = model.fit_generator(train_datagen.flow(training_images, Y_train, batch_size=32),
                              steps_per_epoch=len(training_images) / 32,
                              epochs=10,
                              validation_data=validation_datagen.flow(testing_images, Y_test, batch_size=32),
                              validation_steps=len(testing_images) / 32)



In [ ]:
model.evaluate(testing_images, Y_test, verbose=0)


# Testing Accuracy of Model 


In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()


Confusion Matrix is used for checking results of the model.

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(testing_images)

# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 

# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_test,axis = 1) 

# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(16, 12))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap=sns.cubehelix_palette(8),fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")
plt.title("Confusion Matrix")
plt.show()